In [1]:
import obspy
from pathlib import Path
import datetime
from tqdm import tqdm
import pickle
import numpy as np
import multiprocessing
import matplotlib
import time
import cProfile
matplotlib.use('agg')
one_day = datetime.timedelta(days=1)

In [2]:
input_dir = Path("mseed")
output_dir = Path("data")
if not output_dir.exists():
  output_dir.mkdir()
figure_dir = Path("figure")
if not figure_dir.exists():
  figure_dir.mkdir()

In [3]:
from obspy.clients.fdsn import Client
client = Client("SCEDC")

In [4]:
degree2km = np.pi*6371/180
center = (35.705, -117.504)
horizontal = 0.5
vertical = 0.5

In [5]:
def convert_event(raw):
  event_list = []
  for tmp in raw:
    event_dict = {}
    event_dict['time'] = tmp.origins[0].time
    event_dict['mag'] = tmp.magnitudes[0].mag
    event_dict['mag_type'] = tmp.magnitudes[0].magnitude_type
    event_dict['lat'] = tmp.origins[0].latitude
    event_dict['lng'] = tmp.origins[0].longitude
    event_dict['depth'] = tmp.origins[0].depth / 1000
    event_dict['x'] = (tmp.origins[0].longitude - (center[1]-horizontal))*degree2km
    event_dict['y'] = (tmp.origins[0].latitude - (center[0]-vertical))*degree2km
    event_dict['z'] = tmp.origins[0].depth / 1000
#     print("Event: ", event_dict['x'], event_dict['y'], event_dict['depth'])
    event_list.append(event_dict)
  return event_list

In [6]:
def convert_stream(stream, outfile):
  stream = stream.sort(['network', 'station', 'channel'])
  data_dict = {}
  stations = {}
  events = []
  for trace in stream:
#     print(trace)
    station_id = "{}.{}.{}".format(trace.stats.network, trace.stats.station, trace.stats.location)
    if station_id not in stations:
      
      if len(events) == 0:
        try:
          tmp_event = client.get_events(starttime=trace.stats.starttime, 
                                       endtime=trace.stats.endtime, 
                                       minlatitude=center[0]-vertical,
                                       maxlatitude=center[0]+vertical,
                                       minlongitude=center[1]-horizontal,
                                       maxlongitude=center[1]+horizontal,
                                       catalog="SCEDC")
          events = convert_event(tmp_event)
        except Exception as e:
          print(e)

      stations[station_id] = {}
      tmp_station = obspy.read_inventory("./stations.xml").select(network=trace.stats.network,
                                                                  station=trace.stats.station,
                                                                  location=trace.stats.location)
#       if station_id not in all_stations:
#         tmp_station = client.get_stations(network = trace.stats.network,
#                                    station = trace.stats.station,
#                                    location = trace.stats.location,
#                                    level="response")
#         all_stations[station_id] = tmp_station
#       else:
#         tmp_station = all_stations[station_id]
        
      stations[station_id]['lat'] = tmp_station[0][0].latitude
      stations[station_id]['lng'] = tmp_station[0][0].longitude
      stations[station_id]['x'] = (tmp_station[0][0].longitude - (center[1]-horizontal))*degree2km
      stations[station_id]['y'] = (tmp_station[0][0].latitude - (center[0]-vertical))*degree2km
# #       print("Station: ", stations[station_id]['x'], stations[station_id]['y'])
      stations[station_id]['starttime'] = trace.stats.starttime
      stations[station_id]['endtime'] = trace.stats.endtime
      stations[station_id]['data'] = np.zeros([trace.data.shape[0], 3])
    
    trace = trace.remove_response(inventory=tmp_station, output='ACC')
#     trace.plot(outfile = figure_dir.joinpath(outfile+".{}.{}".format(station_id, trace.stats.channel)+".png"))
#     print(figure_dir.joinpath(outfile+".{}.{}".format(station_id, trace.stats.channel)+".png"))
    if trace.stats.channel[-1] in ['E', '3']:
      stations[station_id]['data'][:, 0] = np.float32(trace.data)
    if trace.stats.channel[-1] in ['N', '2']:
      stations[station_id]['data'][:, 1] = np.float32(trace.data)
    if trace.stats.channel[-1] in ['Z', '1']:
      stations[station_id]['data'][:, 2] = np.float32(trace.data)
  
  data_dict['events'] = events
  data_dict['stations'] = stations
  return data_dict

In [7]:
def remove_short_trace(meta):
  stream = obspy.Stream()
  for trace in meta:
    if trace.stats.endtime - trace.stats.starttime > 12*3600:
      stream.append(trace)
    else:
      print("Remove trace: {}".format(trace))
  return stream

In [8]:
def resample(meta, sampling_rate=100):
  stream = obspy.Stream()
  for trace in meta:
    if trace.stats.sampling_rate == sampling_rate:
      pass
    elif (trace.stats.sampling_rate > sampling_rate) and (np.mod(trace.stats.sampling_rate, sampling_rate) == 0):
      trace.decimate(int(trace.stats.sampling_rate/sampling_rate))
    else:
      print("Sampling rate of {} = {}".format(trace, trace.stats.sampling_rate))
      trace.resample(sampling_rate)
    stream.append(trace)
  return stream

In [9]:
files = sorted(input_dir.glob("*.mseed"))

# for f in tqdm(files):
def process(f):
#   print(str(f))
  if output_dir.joinpath(f.name.rstrip('.mseed')+'.pkl').exists():
#     print("Exist: {}".format(output_dir.joinpath(f.name.rstrip('.mseed')+'.pkl')))
    continue
  else:
    print("Processing: {}".format(str(f)))

  starttime = datetime.datetime.strptime(str(f).split('/')[-1].rstrip('.mseed'), "%Y-%m-%d")
  starttime = obspy.UTCDateTime(starttime)
  meta = obspy.read(str(f))
  meta = meta.detrend('demean')
  meta = meta.merge(fill_value=0)
  meta = remove_short_trace(meta)
#  meta.resample(sampling_rate=100)
  meta = resample(meta)
#   print(meta)
  meta = meta.trim(starttime, starttime+one_day, pad=True, fill_value=0)
  meta = meta.filter("highpass", freq=1.0)
#   meta.plot(outfile=figure_dir.joinpath(f.name.rstrip('.mseed')+'.png'))
  data = convert_stream(meta, outfile=f.name.rstrip('.mseed'))
  with open(output_dir.joinpath(f.name.rstrip('.mseed')+'.pkl'), 'wb') as fp:
    pickle.dump(data, fp)
  print("Write data: ", output_dir.joinpath(f.name.rstrip('.mseed')+'.pkl'))

#   break

100%|██████████| 183/183 [00:00<00:00, 5865.89it/s]


In [ ]:
pool = multiprocessing.Pool(1)
pool.map(process, files)